# Hyperparam tuning 

In [1]:
import os
import glob
import json

import numpy as np

import tensorflow as tf
import tensorflow.keras as keras



from sclouds.helpers import get_lon_array, get_lat_array

# Hyper parameters:

In [2]:
lr = hp.Float(
    'learning_rate',
    min_value=1e-5,
    max_value=1e-2,
    sampling='LOG',
    default=1e-3
)

NameError: name 'hp' is not defined

In [ ]:
filters=hp.Choice('num_filters',
                values=[32, 64, 218],
                default=64,
            )

In [ ]:
filters

In [ ]:
# Custom R2-score metrics for keras backend
def r2_keras(y_true, y_pred):
    import keas.backend as kb
    SS_res =  kb.sum(kb.square(y_true - y_pred)) 
    SS_tot = kb.sum(kb.square(y_true - kb.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + kb.epsilon()) )

def keras_custom_loss_function(y_actual, y_predict):
    """Custum keras loss function, accumulated squared error. """
    import keas.backend as kb
    return np.square(np.subtract(y_actual, y_predict)).sum(axis = 0)

In [ ]:
# TODO figure out how to add the custom loss function later

losses = hp.Choice(
            'loss',
            values=['mean_squared_error', 'mean_absolute_error'],
            default='mean_squared_error',
        ),

In [ ]:
Dense(
    units=hp.Int(
        'units',
        min_value=32,
        max_value=512,
        step=32,
        default=128
    ),
    activation=hp.Choice(
        'dense_activation',
        values=['relu', 'tanh', 'sigmoid'],
        default='relu'
    )
)

In [ ]:
#keras.optimizers.Adam(1e-3)

In [ ]:
hp.Int('units',
        min_value=32,
        max_value=512,
        step=32

In [ ]:
from kerastuner import HyperModel

In [ ]:
HyperModel?

In [ ]:
from kerastuner import HyperModel
from sclouds.helpers import get_lon_array, get_lat_array

n_lon = len(get_lon_array())
n_lat = len(get_lat_array())

class HyperConvLSTM(HyperModel):
    
    DATA_FORMAT        = 'channels_last'
    PADDING            = 'same'
    RETURN_SEQUENCE    = True
    NUM_INPUT_VARS     = 4
    OUTPUT_KERNEL_SIZE = 1
    OUTPUT_FILTER      = 1

    def __init__(self, num_hidden_layers, seq_length):
        self.num_hidden_layers = num_hidden_layers
        self.seq_length = seq_length

    def build(self, hp):
        model =  Sequential()
        # Adding the first layer

        model.add(ConvLSTM2D(filters= hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step=32),                             
                           kernel_size = (3, 3),
                           input_shape = (self.seq_length, n_lat, n_lon, self.NUM_INPUT_VARS),
                           kernel_initializer='glorot_uniform',
                           padding = self.PADDING,
                           return_sequences=self.RETURN_SEQUENCE,
                           data_format=self.DATA_FORMAT))

        #prev_filter = filters[0]
        for i in range(self.num_hidden_layers):
            # Begin with 3D convolutional LSTM layer
            model.add(keras.layers.ConvLSTM2D(filters= hp.Int('filters',
                                                        min_value=32,
                                                        max_value=512,
                                                        step=32),
                                            kernel_size=(3, 3),
                                            kernel_initializer='glorot_uniform',
                                            padding = self.PADDING,
                                            return_sequences = self.RETURN_SEQUENCE,
                                            data_format = self.DATA_FORMAT))
        # Adding the last layer
        model.add(keras.layers.ConvLSTM2D(filters = self.OUTPUT_FILTER,
                                          kernel_size = (1, 1),
                                          kernel_initializer='glorot_uniform',
                                          padding = self.PADDING,
                                          return_sequences = self.RETURN_SEQUENCE,
                                          data_format = self.DATA_FORMAT))

        
        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',
                          values=[1e-2, 1e-3, 1e-4])),
            loss=hp.Choice('loss',
                            values=['mean_squared_error', 'mean_absolute_error'],
                            default='mean_squared_error'),
            metrics=['mean_squared_error', 'mean_absolute_error'])
        
        return model

In [ ]:
#*help(RandomSearch)

# Testing model

In [18]:
from sclouds.helpers import path_convlstm_results

import os
import glob

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization

from sclouds.helpers import path_read_data
from sclouds.io.utils import dataset_to_numpy, dataset_to_numpy_grid, dataset_to_numpy_order, get_xarray_dataset_for_period

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
path_convlstm_results

'/home/hanna/lagrings/results/convlstm/'

In [20]:
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters

hp = HyperParameters()
hypermodel = HyperConvLSTM(num_hidden_layers = 2, seq_length= 4)

tuner = RandomSearch(
        hypermodel,
        objective='mean_squared_error',
        max_trials=10,
        allow_new_entries = True, 
        directory=path_convlstm_results,
        project_name='test_hyperparameters')

In [21]:
from sclouds.io.utils import get_xarray_dataset_for_period, dataset_to_numpy_grid
data = get_xarray_dataset_for_period(start = '2012-01-01', stop = '2012-01-31')
X, y = dataset_to_numpy_grid(data, bias = False)

Num files 5


In [29]:
num_samples, lat, lon, num_vars = X.shape

In [27]:
seq_length = 4

In [28]:
num_samples/seq_length

186.0

In [32]:
#X = X.reshape((int(num_samples/seq_length), seq_length, lat, lon, 4))
y = y.reshape((int(num_samples/seq_length), seq_length, lat, lon, 1))

In [1]:
val_split = 0.2

num_samples, seq_length, _, _, num_vars = X.shape
last_train_idx = int((1-val_split)*num_samples)

X_train = X[:last_train_idx, :, :, :]
y_train = y[:last_train_idx, :,]

X_test = X[last_train_idx:, :, :, :]
y_test = y[last_train_idx:, :,]

NameError: name 'X' is not defined

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Train on 37 samples, validate on 149 samples
Epoch 1/5


In [21]:
models = tuner.get_best_models(num_models=2)

In [22]:
models